In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt
import cv2

In [11]:
TRT_LOGGER = trt.Logger()


def build_engine(onnx_file_path):
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network()
    parser = trt.OnnxParser(network, TRT_LOGGER)
    with open(onnx_file_path, "rb") as model:
        print("Beginning ONNX file parsing")
        parser.parse(model.read())
        print("Completed parsing of ONNX file")
    builder.max_batch_size = 1
    engine = builder.build_cuda_engine(network)
    context = engine.create_execution_context()
    return engine, context


In [13]:
ONNX_FILE_PATH = (
    "/home/user/projects/blog/examples/open_cv/dnn/yolo5/models/yolov5s.onnx"
)
IMAGE_PATH = "/home/user/projects/blog/examples/nvidia/tensorrt_demos/turkish_coffee.jpg"
engine, context = build_engine(ONNX_FILE_PATH)
for binding in engine:
    if engine.binding_is_input(binding):
        input_shape = engine.get_binding_shape(binding)
        input_size = (
            trt.volume(input_shape)
            * engine.max_batch_size
            * np.dtype(np.float32).itemsize
        )
        device_input = cuda.mem_alloc(input_size)
    else:
        output_shape = engine.get_binding_shape(binding)
        host_output = cuda.pagelocked_empty(
            trt.volume(output_shape) * engine.max_batch_size, dtype=np.float32
        )
        device_output = cuda.mem_alloc(host_output.nbytes)
    # Create a stream in which to copy inputs/outputs and run inference.
    stream = cuda.Stream()

    # preprocess input data
    host_input = frame = cv2.imread(IMAGE_PATH)
    cuda.memcpy_htod_async(device_input, host_input, stream)

    # run inference
    context.execute_async(
        bindings=[int(device_input), int(device_output)], stream_handle=stream.handle
    )
    cuda.memcpy_dtoh_async(host_output, device_output, stream)
    stream.synchronize()




Beginning ONNX file parsing
Completed parsing of ONNX file


AttributeError: 'tensorrt.tensorrt.Builder' object has no attribute 'build_cuda_engine'